# Evaluation of experiments - E1
fistly define the UUID of the experiment to use

In [37]:
experimentId="6e8f728f-81ce-4048-8acf-f8d1ffc89a39"

-------------------------------------------------------

In [38]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load mass

In [39]:
samplerate=200  #Should rethink this

In [40]:
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate,onlySpindlesFilteredN2=True)

In [41]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)
annotations.head()

,type,expert,subjectId,labelerId,startTime,duration,samplerate,stopTime,startInd,stopInd
0,spindle,E1,0001,0001,888.327805,0.640579,200,888.968384,177666,177794
1,spindle,E1,0001,0001,905.758061,0.578094,200,906.336155,181152,181267
2,spindle,E1,0001,0001,917.731574,0.847603,200,918.579177,183546,183716
3,spindle,E1,0001,0001,922.078189,0.878845,200,922.957034,184416,184591
4,spindle,E1,0001,0001,939.055445,0.757767,200,939.813212,187811,187963


In [42]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## Load experiment results

In [43]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [44]:
experimentModels.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,9e4dd9bb-a7d9-4ee8-a762-d656d8308d91,0.018639
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,5e1bcb8a-f879-4f42-ae21-0e1ae715095f,0.020055
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,2ecef3ee-0879-42d7-bcf0-ecfa0e3e601a,0.021075
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,52b0d6a8-7f01-4694-85d4-14ef17a025b9,0.018331
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,32c9793a-2115-4e77-8b6f-e8f71d09cecb,0.019222


In [45]:
featureSelection

,characteristic,bandName,window
21,hjortActivity,sigma,0.5
66,petrosian,broadband,0.5
22,hjortActivity,theta,0.5
79,relativePower,beta1,0.5
29,hjortComplexity,sigma,0.5


## Optimal hyperparameter estimation with E1 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [46]:
experimentModels=experimentModels[experimentModels.criteriumName=='E1'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,9e4dd9bb-a7d9-4ee8-a762-d656d8308d91,0.018639
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,5e1bcb8a-f879-4f42-ae21-0e1ae715095f,0.020055
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,2ecef3ee-0879-42d7-bcf0-ecfa0e3e601a,0.021075
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,52b0d6a8-7f01-4694-85d4-14ef17a025b9,0.018331
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,32c9793a-2115-4e77-8b6f-e8f71d09cecb,0.019222
5,0001,E1,[0001],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,241d9889-61ff-47f7-97c2-f502ae3dba38,0.019625
6,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,f6a46037-1f32-40b5-a6ff-cd293b0e4203,0.017824
7,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,6cfc1f74-3653-47eb-a166-f59fefa13825,0.017542
8,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,68c5483d-9072-4478-8a87-a7af3324f690,0.015442
9,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,349fe35c-b749-431d-8e8c-c288b150e998,0.016460


In [47]:
auxThres=pd.DataFrame({
    'hyperThres':[0.3,0.4,0.5,0.6]
})
auxDepth=pd.DataFrame({
    'hyperDepth':[20,30,40,50,60]
})
auxDuration=pd.DataFrame({
    'hyperDuration':[0.3]
})
auxClose=pd.DataFrame({
    'hyperClose':[0.25]
})
auxConfidence=pd.DataFrame({
    'hyperConfidence':[0]
})

hyperParams=pd.merge(auxThres,auxDepth,how='cross')
hyperParams=pd.merge(hyperParams,auxDuration,how='cross')
hyperParams=pd.merge(hyperParams,auxClose,how='cross')
hyperParams=pd.merge(hyperParams,auxConfidence,how='cross')

hyperParams

,hyperThres,hyperDepth,hyperDuration,hyperClose,hyperConfidence
0,0.3,20,0.3,0.25,0
1,0.3,30,0.3,0.25,0
2,0.3,40,0.3,0.25,0
3,0.3,50,0.3,0.25,0
4,0.3,60,0.3,0.25,0
5,0.4,20,0.3,0.25,0
6,0.4,30,0.3,0.25,0
7,0.4,40,0.3,0.25,0
8,0.4,50,0.3,0.25,0
9,0.4,60,0.3,0.25,0


In [48]:
def getCoordAndLenList(labelVector):
    objects=ndi.find_objects(ndi.label(labelVector)[0])
    coords=list(map(lambda x: (x[0].start,x[0].stop,x[0].stop-x[0].start),objects))
    return coords
    
def rawAndProcessedToConfidenceAnnotations(raw,processed,samplerate):
    #calculate the number of true raw outputs in each detection
    labels=ndi.label(processed)
    actualPredicted=np.zeros((labels[1],))
    for i in np.arange(0,labels[1])+1:
        actualPredicted[i-1]=np.sum(raw*(labels[0]==i))
    #calculate the confidences
    outCoords=getCoordAndLenList(processed)
    confidences=[]
    for coords,actual in zip(outCoords,actualPredicted):
        confidences.append(actual/coords[2])
    #create a data frame
    confidenceDf=pd.DataFrame({
        'startInd':[x[0] for x in outCoords],
        'stopInd':[x[1] for x in outCoords],
        'duration':[x[2]/samplerate for x in outCoords],
        'confidence':confidences
    })
    return confidenceDf

In [49]:
def getIou(coordA,coordB):
    if (coordA[1]<coordB[0])|(coordB[1]<coordA[0]): #NOT INTERSECTION
        iou=0
    else:   #INTERSECTION
        inter=np.min((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        union=np.max((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        iou=inter/union
    return iou

def annotationPairToMetrics(annotations,detections,thresIoU=0.2):
    #get the coords
    gtCoords=zip(annotations.startInd,annotations.stopInd)
    outCoords=zip(detections.startInd,detections.stopInd)
    #calculate the iou vector
    iouVector=np.array(list(itt.starmap(getIou,itt.product(gtCoords,outCoords))))
    #reshape to a matrix
    iouMatrix=iouVector.reshape(len(annotations),len(detections))
    #binarize
    binarized=iouMatrix>thresIoU
    #calculateF1
    outF1=(np.sum(np.max(binarized,axis=0))+np.sum(np.max(binarized,axis=1)))/(len(annotations)+len(detections))
    recall=np.sum(np.max(binarized,axis=1))/len(annotations)
    precision=np.sum(np.max(binarized,axis=0))/len(detections)
    return outF1,recall,precision

In [50]:
hyperExperimentModels=[]
hyperMeanF1=[]
hyperStdF1=[]

for ind_hyper,row_hyper in hyperParams.iterrows():
    print(ind_hyper)
    thisExperimentModels=experimentModels.copy()
    hyperThres=row_hyper.hyperThres
    hyperDepth=int(row_hyper.hyperDepth)
    hyperDuration=row_hyper.hyperDuration
    hyperClose=row_hyper.hyperClose
    """ hyperConfidence=row_hyper.hyperConfidence """
    
    meanF1=[]
    meanPrecision=[]
    meanRecall=[]

    stdF1=[]
    stdPrecision=[]
    stdRecall=[]

    for ind,row in thisExperimentModels.iterrows():
        #load model
        model=loadBooster(row.modelId,experimentId,datapath)
        #initialise lists
        rawF1s=[]
        rawPrecisions=[]
        rawRecalls=[]

        f1s=[]
        precisions=[]
        recalls=[]

        #iterate validation subjects
        for valSubjectId in row.val:
            #Define annotations criterium
            usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
            #Load features and labels
            valFeatures=loadFeatureMatrix([valSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
            valLabels=loadLabelsVector([valSubjectId],usedAnnotations,signalsMetadata,samplerate)
            #Predict
            valDMatrix=xgb.DMatrix(data=valFeatures)
            probabilities=model.predict(valDMatrix,iteration_range=(0,hyperDepth))
            raw=probabilities>=hyperThres
            #Processed labels
            processed=labelingProcess(raw,hyperClose,hyperDuration,samplerate)
            """ #Detections with confidence
            detections=rawAndProcessedToConfidenceAnnotations(raw,processed,samplerate)
            #Confidence threshold
            detections=detections[detections.confidence>hyperConfidence].reset_index(drop=True) """
            gtAnnotations=labelVectorToAnnotations(valLabels,samplerate)
            detections=labelVectorToAnnotations(processed,samplerate)
            #Metrics
            f,r,p=annotationPairToMetrics(gtAnnotations,detections)
            
            #Metric appends
            f1s.append(f)
            precisions.append(p)
            recalls.append(r)

        #statistics of the metrics over the subjects of the validation set
        meanF1.append(np.mean(f1s))
        meanPrecision.append(np.mean(precisions))
        meanRecall.append(np.mean(recalls))

        stdF1.append(np.std(f1s))
        stdPrecision.append(np.std(precisions))
        stdRecall.append(np.std(recalls))
        
    thisExperimentModels['meanF1']=meanF1
    thisExperimentModels['meanPrecision']=meanPrecision
    thisExperimentModels['meanRecall']=meanRecall
    
    thisExperimentModels['stdF1']=stdF1
    thisExperimentModels['stdPrecision']=stdPrecision
    thisExperimentModels['stdRecall']=stdRecall

    hyperExperimentModels.append(thisExperimentModels)
    hyperMeanF1.append(np.mean(thisExperimentModels['meanF1']))
    hyperStdF1.append(np.std(thisExperimentModels['meanF1']))
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [51]:
hyperParams['meanMeanF1']=hyperMeanF1
hyperParams['stdMeanF1']=hyperStdF1

In [52]:
optimalInd=hyperParams[hyperParams.meanMeanF1==np.max(hyperParams.meanMeanF1)].index[0]
print("maximal mean score at:")
optimal=hyperParams.iloc[optimalInd]
optimal

maximal mean score at:


hyperThres          0.300000
hyperDepth         20.000000
hyperDuration       0.300000
hyperClose          0.250000
hyperConfidence     0.000000
meanMeanF1          0.684701
stdMeanF1           0.043017
Name: 0, dtype: float64

## Graphical representation of the optimal point

In [53]:
aux=hyperParams[(hyperParams.hyperDepth==optimal.hyperDepth)&(hyperParams.hyperDuration==optimal.hyperDuration)&
                (hyperParams.hyperClose==optimal.hyperClose)&(hyperParams.hyperConfidence==optimal.hyperConfidence)].reset_index(drop=True)
px.scatter(aux,x='hyperThres',y='meanMeanF1',error_y='stdMeanF1')

In [54]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperDuration==optimal.hyperDuration)&
                (hyperParams.hyperClose==optimal.hyperClose)&(hyperParams.hyperConfidence==optimal.hyperConfidence)].reset_index(drop=True)
px.scatter(aux,x='hyperDepth',y='meanMeanF1',error_y='stdMeanF1')

In [55]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperDepth==optimal.hyperDepth)&
                (hyperParams.hyperClose==optimal.hyperClose)&(hyperParams.hyperConfidence==optimal.hyperConfidence)].reset_index(drop=True)
px.scatter(aux,x='hyperDuration',y='meanMeanF1',error_y='stdMeanF1')

In [56]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperDepth==optimal.hyperDepth)&
                (hyperParams.hyperDuration==optimal.hyperDuration)&(hyperParams.hyperConfidence==optimal.hyperConfidence)].reset_index(drop=True)
px.scatter(aux,x='hyperClose',y='meanMeanF1',error_y='stdMeanF1')

In [57]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperDepth==optimal.hyperDepth)&
                (hyperParams.hyperDuration==optimal.hyperDuration)&(hyperParams.hyperClose==optimal.hyperClose)].reset_index(drop=True)
px.scatter(aux,x='hyperConfidence',y='meanMeanF1',error_y='stdMeanF1')